In [1]:
using Pkg 
Pkg.instantiate()
using qAlgebra

Precompiling project...
    952.5 ms  ✓ qAlgebra
  1 dependency successfully precompiled in 1 seconds. 7 already precompiled.


In [2]:
qspace = StateSpace("alpha", "beta(t)", "gamma_i", "delta_i", operators=["A(!i)", "B(U,H,i)"], h=QubitPM(), i=(3, QubitPauli()), b=Ladder())

StateSpace: [α, β(t), γᵢ, γⱼ, γₖ, δᵢ, δⱼ, δₖ]
   - SubSpace ["h"]: PM Qubit (Fermionic):  pₚ, mₚ, zₚ, Iₚ (identity)
   - SubSpace ["i", "j", "k"]: Pauli Qubit (Fermionic):  xₚ, yₚ, zₚ, Iₚ (identity)
   - SubSpace ["b"]: Ladder (Bosonic):  p†, p
   - Op: A
   - Op: B(H,U)


In [3]:
base_operators(qspace)
base_operators("alpha", qspace)
base_operators("i", qspace)
base_operators("h", qspace)
base_operators("j", qspace)
base_operators("I", qspace)
base_operators("A", qspace)
println("Done") 

Done


In [ ]:
function *(Q1::qComposite, Q2::qExpr)::qComposite
    Q_new = copy(Q1)
    Q_new.expr = Q1.expr * Q2
    return Q_new
end

function *(Q1::qExpr, Q2::qComposite)::qExpr
    Q_new = copy(Q1)
    Q_new.expr = Q1.expr * Q2
    return Q_new
end

function *(Q1::qComposite, num::Number)::qComposite
    Q_new = copy(Q1)
    Q_new.expr = Q1.expr * num
    return Q_new
end

function *(num::Number, Q2::qComposite)::qComposite
    return Q2 * num
end

In [ ]:
"""
    pad_before_qAbstracts(p::qProd) -> qProd

Insert a neutral `qTerm` *before every `qAbstract`* that appears before
the final `qTerm`. Ensures qTerm shifts don't affect qAbstract positions.
"""
function pad_before_qAbstracts(p::qProd)::qProd
    ss = p.statespace
    neutral = qTerm(ss.neutral_op)

    # Find the last qAtom position
    last_atom_index = findlast(t -> isa(t, qTerm), p.expr)

    # If there are no qAtoms, no padding is necessary
    if last_atom_index === nothing
        return copy(p)
    end

    terms = copy(p.expr)
    # Iterate in reverse to keep indices stable while inserting
    for i in last_atom_index-1:-1:2
        both_qAbstract = isa(terms[i], qAbstract) && isa(terms[i-1], qAbstract)
        if both_qAbstract
            insert!(terms, i, copy(neutral))
        end
    end
    # Special case: if the first element is a qAbstract
    if isa(terms[1], qAbstract)
        insert!(terms, 1, copy(neutral))
    end
    return qProd(ss, copy(p.coeff_fun), terms)
end

function qTerms2left(p::qProd)::Vector{qProd}
    ss = p.statespace
    coeff_fun = p.coeff_fun
    last_atom_index = findlast(t -> isa(t, qTerm), terms)
    all_terms::Vector{Vector{qAtom}} = [copy(p.expr)]
    all_coeffs::Vector{ComplexRational} = [one(ComplexRational)]
    # for storing intermediate creations 
    new_all_terms::Vector{Vector{qAtom}} = []
    new_all_coeffs::Vector{ComplexRational} = []

    # Find the last qAtom position
    last_atom_index = findlast(t -> isa(t, qTerm), terms)
    for i in last_atom_index:-2:3
        for k in 1:length(all_terms)
            terms = all_terms[k]
            coeff = all_coeffs[k]
            curr_qterm = terms[i]
            curr_qabstract = terms[i-1]
            c_abstract = where_acting(curr_qabstract, statespace) # boolean vector
            c_term = where_acting(curr_qterm, statespace)  # boolean vector
            if !any(c_term) 
                deleteat!(terms, i)
                push!(new_terms, term)
                push!(new_coeffs, copy(coeff))
            elseif all([nand(a,b) for (a,b) in zip(c_abstract, c_term)])   # commutes 
                prev_qterm = term[i-2]
                new_terms, new_coeffs = multiply_qterm(prev_qterm, curr_qterm, statespace)
                # construct new terms 
                deleteat!(terms, i)
                for (t,c) in zip(new_terms, new_coeffs)
                    new_term = copy(terms)
                    new_term[i-2] = t 
                    push!(new_terms, new_term)
                    push!(new_coeffs, coeff*c)
                end
            else 
                # split curr_qterm into 2, one that commutes with curr_qabstract and one that doesn't
                commuting_qterm = copy(ss.neutral_op)
                non_commuting_qterm = copy(ss.neutral_op)
                # add at indexes that aren't in c_abstract
                for (i, b, q_ind) in enumerate(c_abstract, curr_qterm.op_indices)
                    if b
                        non_commuting_qterm[i] = q_ind
                    else
                        commuting_qterm[i] = q_ind
                    end
                end
                prev_qterm = term[i-2]
                new_terms, new_coeffs = multiply_qterm(qTerm(prev_qterm), qTerm(commuting_qterm), statespace)
                terms[i] = qTerm(non_commuting_qterm)
                for (t, c) in zip(new_terms, new_coeffs)
                    new_term = copy(terms)
                    new_terms[i-2] = t 
                    push!(new_terms, new_term)
                    push!(coeffs, coeff*c)
                end
            end
        end
        all_terms = copy(new_terms)
        all_coeffs = copy(new_coeffs)
        new_terms = Vector{qAtom}[]
        new_coeffs = Vector{ComplexRational}[]
    end
    if length(p.expr) > 0
        for t in all_terms
            if is_numeric(t.expr[1], ss)
                deleteat!(t.expr, 1)
            end
        end
    end
    # create qProd for each 
    return qProd[qProd(ss, c, t) for (c,t) in zip(all_coeffs, all_terms)]
end

function reduce_qabstractpairs(p::qProd)::Tuple{qProd, Bool}
    # remove pairs of qAbstract if possible 
    ss = p.statespace
    terms = copy(p.expr)
    i = 1
    did_any = false
    while i < length(terms)
        did_this = false
        if is_qAbstract(terms[i]) && is_qAbstract(terms[i+1])
            # check if they are of the same subtypes
            if terms[i].key_index == terms[i+1].key_index && terms[i].sub_index == terms[i+1].sub_index && terms[i].index_map == terms[i+1].index_map
                if xor(terms[i].dag, terms[i+1].dag) 
                    if terms[i].operator_type.hermitian 
                        new_term = copy(terms[i])
                        new_term.dag = false 
                        new_term.exponent += terms[i+1].exponent
                        did_this = true
                    elseif terms[i].operator_type.unitary
                        new_term = copy(terms[i])
                        new_term.dag = false 
                        new_term.exponent = terms[i].exponent*(-1)^terms[i].dag + terms[i+1].exponent*(-1)^terms[i+1].dag
                        did_this = true
                    end
                elseif nor(terms[i].dag, terms[i+1].dag)  # neither dag 
                    new_term = copy(terms[i])
                    new_term.dag = false 
                    new_term.exponent += terms[i+1].exponent
                    did_this = true
                end
            end
            if did_this
                deleteat!(terms, i+1)
                terms[i] = new_term
                did_any = true
            else
                i += 1
            end
        end
    end
    return qProd(ss, p.coeff_fun, terms), did_any
end

function simplify(p::qProd)::qExpr
    p_new = qProd[pad_before_qAbstracts(p)]
    did_any = true 
    while did_any
        left_terms = qProd[] 
        for curr_p in p_new 
            append!(left_terms, qTerms2left(curr_p)) 
        end 
        did_any = false 
        p_new = qProd[] 
        
        for i in 1:length(left_terms)
            new_term, did_it = reduce_qabstractpairs(left_terms[i])
            push!(p_new, new_term)
            if did_it
                did_any = true
            end
        end
    end
    return qExpr(p.statespace, p_new)
end             

2-element BitVector:
 1
 0

In [3]:
.-[1,2]

2-element Vector{Int64}:
 -1
 -2

In [4]:
function term_equal_indexes(abstract::qAbstract, index1::Int, index2::Int, subspace::SubSpace)::Tuple{Bool, Vector{qAbstract}}
end

UndefVarError: UndefVarError: `qAbstract` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [5]:
term("alpha*A*xi*yi*b'*B*ph*mh")
println("Done")

Done


In [ ]:
function non_trivial(q::qAtom, index::

In [ ]:
function term_equal_indexes(prod::qProd, index1::Int, index2::Int, subspace::SubSpace)::Tuple{Bool, Vector{qProd}}
    changed_any = false
    term_variants = Vector{Vector{qAtom}}()
    for atom in prod.expr
        changed, variants = term_equal_indexes(atom, index1, index2, subspace)
        push!(term_variants, variants)
        changed_any |= changed  # Check if any term was changed
    end

    if !changed_any
        return false, [prod]
    end

    # Generate all combinations (cartesian product) of updated terms
    combinations = Iterators.product(term_variants...)

    simplified_products = qProd[]

    for combo in combinations
        new_expr = collect(combo)
        new_prod = qProd(prod.coeff_fun, new_expr)
        push!(simplified_products, simplify(new_prod))
    end

    return true, simplified_products
end


UndefVarError: UndefVarError: `custom_sort_key` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [2]:
using ComplexRationals
a = ComplexRational(1,0,1)
copy(a)

1

In [ ]:
expr = 2 * alpha * im * xi + alpha * Dag(b) * xi * yi

In [ ]:
qsum = Sum("j",  alpha*yi*yj+Sum("k", beta*alpha^2*xi*xj*xk))

In [ ]:
flat_sum = flatten(qsum)

In [ ]:
# The sum still covers all combinations of indexes j,k
# We can transform it into a neq sum, in which the indexes j and k are distinct. the following function then expands into all possible cases
neq_sum = neq(qsum) # this also flattens the sum

In [ ]:
# A differential equation of expectation values can be constructed via
dzi_dt = d_dt(zi, alpha*expr+qsum)

In [ ]:
clipboard(string(dzi_dt))

In [ ]:
clipboard(latex_string(dzi_dt))